In [1]:
import folium
import pandas as pd

In [2]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [3]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

In [4]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
map_center = [30, -100]
site_map = folium.Map(location=map_center, zoom_start=4)
site_map

# **Task 1: Mark all launch sites on a map**

In [7]:
for index, row in launch_sites_df.iterrows():


    coordinates = [row['Lat'], row['Long']]

    circle = folium.Circle(coordinates, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(f"{row['Launch Site']}"))


    marker = folium.map.Marker(
    coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % f"{row['Launch Site']}",
        )
    )

    site_map.add_child(circle)
    site_map.add_child(marker)


In [8]:
site_map

# **Task 2: Mark the success/failed launches for each site on the map**

In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [10]:
marker_cluster = MarkerCluster()

In [11]:
def set_marker_color(row):
    if row['class'] == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df.apply(set_marker_color, axis=1)

In [12]:
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [13]:
for index, record in spacex_df.iterrows():
    marker_color = record['marker_color']
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=record['Launch Site'],
        icon=folium.Icon(color=marker_color)
    )

    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)

site_map

# **TASK 3: Calculate the distances between a launch site to its proximities**

In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [16]:
launch_site_lat = 28.573255
launch_site_lon = -80.646895

In [18]:
railway_lat= 28.57266
railway_lon= -80.65391

highway_lat= 28.57341
highway_lon= -80.65545

coastline_lat= 28.57163
coastline_lon= -80.60693

city_lat= 28.58294
city_lon= -80.65088

In [19]:
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

In [20]:
print(distance_railway, distance_highway, distance_coastline, distance_city)

0.6884349123372359 0.8358530783513737 3.9081029856774423 1.1454278594141407


In [22]:
railway_coordinates = [28.57266, -80.65391]
highway_coordinates = [28.57341, -80.65545]
coastline_coordinates = [28.57163, -80.60693]
city_coordinates = [28.58294, -80.65088]

In [21]:
launch_site_coordinates = [28.573255, -80.646895]
railway_coordinates = [28.57266, -80.65391]
coordinates = [launch_site_coordinates, railway_coordinates]

lines = folium.PolyLine(
    locations=coordinates,
    weight=1,
    color='blue',
    opacity=0.7
)

site_map.add_child(lines)

site_map

In [23]:
coordinates = [launch_site_coordinates, highway_coordinates]

lines = folium.PolyLine(
    locations=coordinates,
    weight=1,
    color='blue',
    opacity=0.7
)

site_map.add_child(lines)

coordinates = [launch_site_coordinates, coastline_coordinates]

lines = folium.PolyLine(
    locations=coordinates,
    weight=1,
    color='blue',
    opacity=0.7
)

site_map.add_child(lines)

coordinates = [launch_site_coordinates, city_coordinates]

lines = folium.PolyLine(
    locations=coordinates,
    weight=1,
    color='blue',
    opacity=0.7
)

site_map.add_child(lines)
site_map

In [26]:
distance_marker = folium.Marker(
   railway_coordinates,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM \nNearest Railway".format(distance_railway),
       )
   )

site_map.add_child(distance_marker)

distance_marker = folium.Marker(
   highway_coordinates,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM \nNearest Highway".format(distance_highway),
       )
   )

site_map.add_child(distance_marker)

distance_marker = folium.Marker(
   coastline_coordinates,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM \nNearest Coastline".format(distance_coastline),
       )
   )

site_map.add_child(distance_marker)

distance_marker = folium.Marker(
   city_coordinates,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM \nNearest City".format(distance_city),
       )
   )

site_map.add_child(distance_marker)

site_map

In [25]:
site_map.save('launch_site_to_railway_map.html')